In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import csv

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/soda-inspection/sample_submission.csv
/kaggle/input/soda-inspection/train.csv
/kaggle/input/soda-inspection/test.csv
/kaggle/input/soda-inspection/test/test_64.jpg
/kaggle/input/soda-inspection/test/test_42.jpg
/kaggle/input/soda-inspection/test/test_5.jpg
/kaggle/input/soda-inspection/test/test_34.jpg
/kaggle/input/soda-inspection/test/test_25.jpg
/kaggle/input/soda-inspection/test/test_22.jpg
/kaggle/input/soda-inspection/test/test_59.jpg
/kaggle/input/soda-inspection/test/test_23.jpg
/kaggle/input/soda-inspection/test/test_7.jpg
/kaggle/input/soda-inspection/test/test_28.jpg
/kaggle/input/soda-inspection/test/test_17.jpg
/kaggle/input/soda-inspection/test/test_49.jpg
/kaggle/input/soda-inspection/test/test_9.jpg
/kaggle/input/soda-inspection/test/test_50.jpg
/kaggle/input/soda-inspection/test/test_54.jpg
/kaggle/input/soda-inspection/test/test_36.jpg
/kaggle/input/soda-inspection/test/test_18.jpg
/kaggle/input/soda-inspection/test/test_43.jpg
/kaggle/input/soda-inspect

In [2]:
def detect_content_high(image):
    # Definindo as coordenadas e dimensões da região de interesse
    region_x = 30
    region_y = 113
    region_width = 70
    region_height = 22

    # Extraindo a região de interesse da imagem
    content_region = image[region_y:region_y+region_height, region_x:region_x+region_width]

    # Definindo os limites de cor para identificar o conteúdo alto
    lower_color = np.array([0, 0, 0])
    upper_color = np.array([50, 50, 50])

    # Criando uma máscara para identificar a presença de conteúdo alto
    mask = cv2.inRange(content_region, lower_color, upper_color)

    # Verificando se a máscara possui algum pixel indicando presença de conteúdo alto
    has_high_content = np.any(mask == 255)

    # Retornando 1 se houver conteúdo alto e 0 caso contrário
    return 1 if has_high_content else 0

In [3]:
def detect_content_low(image):
    # Definindo as coordenadas e dimensões da região de interesse
    region_x = 30
    region_y = 130
    region_width = 70
    region_height = 22

    # Extraindo a região de interesse da imagem
    content_region = image[region_y:region_y+region_height, region_x:region_x+region_width]

    # Definindo os limites de cor para identificar o conteúdo baixo
    lower_color = np.array([0, 0, 0])
    upper_color = np.array([60, 60, 60])

    # Criando uma máscara para identificar a presença de conteúdo baixo
    mask = cv2.inRange(content_region, lower_color, upper_color)

    # Verificando se a máscara possui algum pixel preto (representando a ausência de conteúdo baixo)
    has_low_content = np.any(mask == 255)

    # Retornando 0 se não houver conteúdo baixo e 1 caso contrário
    return 0 if has_low_content else 1


In [4]:
def detect_cover_none(image):
    # Definindo as coordenadas e dimensões da região da tampa
    region_x = 0
    region_y = 0
    region_width = 100
    region_height = 70

    # Extraindo a região da tampa da imagem
    content_region = image[region_y:region_y+region_height, region_x:region_x+region_width]

    # Definindo os limites de cor para identificar a ausência de tampa
    lower_color = np.array([100, 0, 0])
    upper_color = np.array([255, 60, 60])

    # Criando uma máscara para identificar a ausência de tampa
    mask = cv2.inRange(content_region, lower_color, upper_color)

    # Verificando se a máscara possui algum pixel vermelho (indicando a presença de tampa)
    has_red = np.any(mask == 255)

    # Retornando 1 se não houver tampa e 0 caso contrário
    return 0 if has_red else 1


In [5]:
def detect_bottle_smashed(image):
    height, width, _ = image.shape
    third_length = width // 3

    image = image[:, third_length-5:2*third_length+20]
    image = image[100:280, :]

    # Convertendo a imagem para o espaço de cores RGBA
    image_rgba = cv2.cvtColor(image, cv2.COLOR_BGR2RGBA)

    # Definindo os valores de cor mínimos e máximos
    lower_color = np.array([120, 70, 38, 255], dtype=np.uint8)  # Menor valor de cada componente de cor (R, G, B, A)
    upper_color = np.array([160, 130, 100, 255], dtype=np.uint8)  # Maior valor de cada componente de cor (R, G, B, A)

    # criando as máscaras inferior e superior
    mask = cv2.inRange(image_rgba, lower_color, upper_color)

    # Excluindo a região central da imagem da máscara
    height, width, _ = image.shape
    center_x, center_y = width // 2, height // 2
    radius = min(center_x, center_y) // 2  # Raio do círculo centrado na imagem
    cv2.circle(mask, (center_x, center_y), radius, 0, -1)  # Desenhar um círculo preto na máscara

    # Aplicando as máscaras à imagem original
    masked_image = cv2.bitwise_and(image, image, mask=mask)

    # Calculando a quantidade de pixels na área de interesse
    num_pixels = np.sum(mask)
    
    # Verificando se a quantidade de pixels é maior que 260000, se for é porque a garrafa ta amassada
    return 1 if num_pixels > 202000 else 0

In [6]:
def detect_label_white(image):
    # Definindo as coordenadas e dimensões da região da etiqueta
    region_x = 30
    region_y = 175
    region_width = 130
    region_height = 70

    # Extraindo a região da etiqueta da imagem
    label_region = image[region_y:region_y+region_height, region_x:region_x+region_width]

    # Convertendo a região para escala de cinza
    gray_image = cv2.cvtColor(label_region, cv2.COLOR_RGB2GRAY)

    # Aplicando limiar adaptativo à imagem em escala de cinza
    threshold = cv2.adaptiveThreshold(gray_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

    # Calculando a porcentagem de pixels brancos na imagem limiarizada
    total_pixels = np.prod(threshold.shape)
    white_pixels = np.sum(threshold == 255)
    white_percentage = (white_pixels / total_pixels) * 100

    # Verificando se a área branca na etiqueta é significativa
    big_white_area = white_percentage > 75

    # Retornando 1 se a área branca for significativa e 0 caso contrário
    return 1 if big_white_area else 0

In [7]:
def detect_label_misplaced(image):
    # Definindo as coordenadas e dimensões da região da etiqueta
    region_x = 5
    region_y = 150
    region_width = 150
    region_height = 85

    # Extraindo a região da imagem
    label_region = image[region_y:region_y+region_height, region_x:region_x+region_width]

    # Aplicando desfoque gaussiano à região da etiqueta
    blurred_label_region = cv2.GaussianBlur(label_region, (5, 5), 0)

    # Convertendo a região da etiqueta para escala de cinza
    gray_label_region = cv2.cvtColor(blurred_label_region, cv2.COLOR_BGR2GRAY)

    # Equalizando o histograma da imagem em escala de cinza
    equalized = cv2.equalizeHist(gray_label_region)

    # Aplicando operações de morfologia para fechar buracos na imagem
    kernel = np.ones((5, 5), np.uint8)
    closing = cv2.morphologyEx(equalized, cv2.MORPH_CLOSE, kernel)

    # Aplicando o operador Sobel para detectar bordas na imagem fechada
    sobel_x = cv2.Sobel(closing, cv2.CV_16S, 1, 0)
    sobel_y = cv2.Sobel(closing, cv2.CV_16S, 0, 1)

    # Detecção de bordas com Canny
    edges = cv2.Canny(sobel_x, sobel_y, 110, 255)

    # Detecção de linhas usando a transformada de Hough
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=45, minLineLength=55, maxLineGap=10)

    # Verificando se há linhas detectadas e se alguma linha tem inclinação inadequada
    if lines is not None:
        # Inicializa a flag indicando se a etiqueta está deslocada
        is_misplaced = False

        # Itera sobre as linhas detectadas
        for line in lines:
            # Extrai as coordenadas dos pontos da linha
            x1, y1, x2, y2 = line[0]

            # Calcula o ângulo da linha em relação ao eixo x
            angle = np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi

            # Verifica se o ângulo está dentro do intervalo adequado
            if np.abs(angle) > 5 and np.abs(angle) < 20:
                # Se encontrar uma linha com o ângulo adequado, define a flag como True e sai do loop
                is_misplaced = True
                break

        # Retorna 1 se a etiqueta estiver deslocada, caso contrário retorna 0
        return int(is_misplaced)
    else:
        return 0

In [8]:
def detect_label_none(image):
    # Define as coordenadas e dimensões da região da etiqueta na imagem
    region_x = 20
    region_y = 175
    region_width = 150
    region_height = 150

    # Extrai a região da etiqueta da imagem
    label_region = image[region_y:region_y + region_height, region_x:region_x + region_width]

    # Converte a região da etiqueta para o espaço de cores HSV
    label_hsv = cv2.cvtColor(label_region, cv2.COLOR_BGR2HSV)

    # Define os intervalos de cor para o vermelho
    lower_red = np.array([0, 100, 100])
    upper_red = np.array([10, 255, 255])

    # Cria uma máscara para os pixels vermelhos na imagem HSV
    red_mask = cv2.inRange(label_hsv, lower_red, upper_red)

    # Aplica a máscara à região da etiqueta
    masked_region = cv2.bitwise_and(label_region, label_region, red_mask)

    # Verifica se há pixels vermelhos na região da etiqueta
    red_present = np.any(masked_region == 255)

    # Retorna 0 se pixels vermelhos estiverem presentes (indicando que há uma etiqueta), caso contrário retorna 1
    return 0 if red_present else 1

In [9]:
def detect_bottle_none(image):
    # Converte a imagem para escala de cinza
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Define as coordenadas e dimensões da região central da imagem
    region_x = int(image.shape[1] / 4)
    region_y = int(image.shape[0] / 4)
    region_width = int(image.shape[1] / 2)
    region_height = int(image.shape[0] / 2)

    # Extrai a região central da imagem em escala de cinza
    central_region = gray_image[region_y:region_y + region_height, region_x:region_x + region_width]

    # Detecta as bordas na região central
    edges = cv2.Canny(central_region, 100, 200)

    # Verifica se há bordas na região central
    has_edges = np.any(edges > 0)

    # Retorna 1 se não houver bordas na região central (indicando a ausência de uma garrafa), caso contrário retorna 0
    return 1 if not has_edges else 0


In [10]:
def write_csv(filename, data):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        # Escrever o cabeçalho
        writer.writerow(["id", "CONTENT_HIGH", "CONTENT_LOW", "COVER_NONE", "BOTTLE_SMASHED", "LABEL_WHITE", "LABEL_MISPLACED", "LABEL_NONE", "BOTTLE_NONE"])
        # Escrever os dados
        writer.writerows(data)

In [11]:
def identify_image_features(image, defects):
    x = 105
    y = 0
    width = 135
    height = 300

    # Separando o meio da imagem
    bottle = image[y:y+height, x:x+width]

    # Convertendo para RGB
    bottle = cv2.cvtColor(bottle, cv2.COLOR_BGR2RGB)

    # Criando uma copia do dicionario de defeitos (todo zerado)
    defects_values = defects.copy()

    # Chamando as funções de cada defeito
    is_none = detect_bottle_none(bottle)
    if is_none == 1:
        defects_values["BOTTLE_NONE"] = is_none
    else:
        defects_values["CONTENT_HIGH"] = detect_content_high(bottle)
        defects_values["CONTENT_LOW"] = detect_content_low(bottle)
        defects_values["COVER_NONE"] = detect_cover_none(bottle)
        defects_values["LABEL_WHITE"] = detect_label_white(bottle)
        defects_values["LABEL_MISPLACED"] = detect_label_misplaced(bottle)
        defects_values["LABEL_NONE"] = detect_label_none(bottle)
        defects_values["BOTTLE_SMASHED"] = detect_bottle_smashed(image)

    # Criando array com resultados a partir do dicionário
    result = np.array(list(defects_values.values()))

    return result

In [12]:
defects = {
    "CONTENT_HIGH": 0,
    "CONTENT_LOW": 0,
    "COVER_NONE": 0,
    "BOTTLE_SMASHED": 0,
    "LABEL_WHITE": 0,
    "LABEL_MISPLACED": 0,
    "LABEL_NONE": 0,
    "BOTTLE_NONE": 0
}

data = []

print("Result")

# Obtendo as imagens do dataset
for i in range (1,65):
    image = cv2.imread(f'/kaggle/input/soda-inspection/test/test_{i}.jpg')
    print(f'test_{i}.jpg')

    result = identify_image_features(image, defects)
    print(result)

    data.append([f'test_{i}.jpg'] + list(result))

write_csv('submission.csv', data)

Result
test_1.jpg
[1 0 0 0 0 0 0 0]
test_2.jpg
[0 0 0 0 0 0 0 0]
test_3.jpg
[0 0 0 0 0 0 0 0]
test_4.jpg
[0 0 0 0 1 0 0 0]
test_5.jpg
[0 1 0 0 0 0 0 0]
test_6.jpg
[1 0 0 0 0 0 0 0]
test_7.jpg
[1 0 0 0 0 0 0 0]
test_8.jpg
[0 0 0 0 0 0 0 0]
test_9.jpg
[0 0 0 0 0 0 0 0]
test_10.jpg
[1 0 0 0 0 0 0 0]
test_11.jpg
[0 0 0 0 0 0 0 0]
test_12.jpg
[0 0 0 0 0 0 0 0]
test_13.jpg
[0 0 0 0 0 0 0 0]
test_14.jpg
[1 0 0 0 0 0 0 0]
test_15.jpg
[0 0 0 0 0 0 0 0]
test_16.jpg
[0 0 0 0 0 0 1 0]
test_17.jpg
[1 0 0 0 0 0 0 0]
test_18.jpg
[0 0 0 0 0 0 0 0]
test_19.jpg
[1 0 0 1 0 0 0 0]
test_20.jpg
[0 0 0 0 1 0 0 0]
test_21.jpg
[0 0 0 0 0 0 0 0]
test_22.jpg
[1 0 0 0 0 1 0 0]
test_23.jpg
[0 0 0 0 0 0 0 1]
test_24.jpg
[0 0 0 0 0 0 0 1]
test_25.jpg
[1 0 1 0 0 0 0 0]
test_26.jpg
[0 0 0 0 0 0 0 0]
test_27.jpg
[0 0 0 0 0 0 1 0]
test_28.jpg
[1 0 0 1 0 1 0 0]
test_29.jpg
[0 0 0 0 0 0 0 0]
test_30.jpg
[0 0 0 0 0 0 0 0]
test_31.jpg
[1 0 1 0 0 0 0 0]
test_32.jpg
[0 0 0 0 0 0 0 0]
test_33.jpg
[0 0 0 0 0 0 0 0]
test_34.jpg
